# Introduction
Streamlined version of 02_end2end_MLLearning.ipynb

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import tarfile
import urllib.request
import matplotlib.pyplot as plt
import time
import pdb
import joblib, os, pickle
from scipy import stats
from datetime import datetime

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector, make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

from MLTools import ClusterSimilarity, column_ratio, ratio_name, AgeSimilarity, load_housing_data

# Data Import

In [3]:
housing = load_housing_data()

datasets/housing/housing.csv


# Train Test Split

## Discretization

In [4]:
# housing["income_cat"] = pd.qcut(housing.median_income, 5, labels = [1, 2, 3, 4, 5])
# housing.drop("income_cat", axis=1, inplace=True)
housing_income_cat = pd.qcut(housing.median_income, 5, labels = [1, 2, 3, 4, 5])

In [5]:
strat_train_set, strat_test_set = train_test_split(housing, test_size=0.2, stratify=housing_income_cat, random_state=42)
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

## Pipelines

### Ratio Pipeline
This is a very simple pipeline/function which encompass all operations necessary to compute ratio operations (such as defining _rooms\_per\__house_ previously)

In [6]:
ratio_pipeline = Pipeline([    
    ("impute", SimpleImputer(strategy="median")),
    ("computeRatio", FunctionTransformer(column_ratio, feature_names_out=ratio_name)),
    ("standardize", StandardScaler())
])

### Logarithm Pipeline
Features which present long tails to the right, and therefore need to be transformed in order to be more symmetric.

This will be done by replacing these features with their logarithm.

In [7]:
log_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("computeLog", FunctionTransformer(np.log, feature_names_out="one-to-one")), # Don't ask me WHY feature_names_out must be "one-to-one", but it crashes if it's anything else!
    ("standardize", StandardScaler())
])

### Geography Pipeline
As mentioned earlier, contextualizing the latitude and longitude in their current form is difficult for experts and machines, therefore we want to transform them to simplify this.

The book offers a great idea in which we compute clusters/hotspots of house values, then we compute the distance of each geographic coordinates to the nearest cluster!
I am hopeful I would have thought of the base idea, but I am not sure I'd have been able to implement it. But since I'm here to learn, I will shamelessly steal their implementation.

The first step of this pipeline is to compute k-means clusters of the two geographic coordinates we pass in arguments (bunch up together the x and y coordinates and finds the closest clusters).

Then, we use _rbf\_kernel_ to compute the similarity (according to ~all (non-geographic) features (income, age, total\_rooms, etc.))~ the geographic distance between each district (rows in the _housing_ table) and each cluster center.
Therefore, the output of this will be a matrix with the same number of rows as _housing_ and 1 column per cluster center.

In [8]:
cluster_simil = ClusterSimilarity(n_clusters=10, gamma=1, random_state=42)

### Categorical Pipeline
The variable _ocean\_proximity_ is a categorical, and to allow correct processing by our algorithms, we will transform it into numerical. But to avoid "ordering" them, we will use a binary attribute per category, so that "proximity" in number (0: 'Near Bay', 1: '>2H Ocean', for example) is not interpreted as a proximity in value. To do so, we will use OneHotEncoder.

In [9]:
cat_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy="most_frequent")),
    ("OneHotEncoder", OneHotEncoder(handle_unknown="ignore")),
])

### Default Numerical Pipeline

In [10]:
default_num_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("standardize", StandardScaler()),
])

### Age - K-Means + rbf_kernel

*Let's start again from scratch*.
Let's fit a gaussian distribution around 39 and 17 (these values were obtained previously thanks to k-means clustering), and then somehow, for each point, compute the similarity to each 

In [11]:
age_simil = ClusterSimilarity(n_clusters=2, random_state=42)
similarities = age_simil.fit_transform(housing[["housing_median_age"]])

### Complete Transformer

In [12]:
preprocessing = ColumnTransformer([
    ("bedrooms", ratio_pipeline, ["total_bedrooms", "total_rooms"]),
    ("rooms_per_house", ratio_pipeline, ["total_rooms", "households"]),
    ("people_per_house", ratio_pipeline, ["population", "households"]),
    ("log", log_pipeline, ["total_bedrooms", "total_rooms", "population", "households", "median_income"]),
    ("geo", cluster_simil, ["latitude", "longitude"]),
    ("cat", cat_pipeline, make_column_selector(dtype_include=object)),
    ("age", age_simil, ["housing_median_age"]),
], remainder=default_num_pipeline)

In [13]:
housing_prepared = preprocessing.fit_transform(housing)
housing_prepared.shape

(16512, 25)

In [14]:
preprocessing.get_feature_names_out()

array(['bedrooms__ratio', 'rooms_per_house__ratio',
       'people_per_house__ratio', 'log__total_bedrooms',
       'log__total_rooms', 'log__population', 'log__households',
       'log__median_income', 'geo__Cluster 0 similarity',
       'geo__Cluster 1 similarity', 'geo__Cluster 2 similarity',
       'geo__Cluster 3 similarity', 'geo__Cluster 4 similarity',
       'geo__Cluster 5 similarity', 'geo__Cluster 6 similarity',
       'geo__Cluster 7 similarity', 'geo__Cluster 8 similarity',
       'geo__Cluster 9 similarity', 'cat__ocean_proximity_<1H OCEAN',
       'cat__ocean_proximity_INLAND', 'cat__ocean_proximity_ISLAND',
       'cat__ocean_proximity_NEAR BAY', 'cat__ocean_proximity_NEAR OCEAN',
       'age__Cluster 0 similarity', 'age__Cluster 1 similarity'],
      dtype=object)

## Select and Train a Model

### Random Forest Regressor

In [15]:
forest_reg = Pipeline([
    ("preprocessing", preprocessing),
    ("ForestReg", RandomForestRegressor(random_state=42)),
])
forest_reg.fit(housing, housing_labels)
housing_predictions_forestReg = forest_reg.predict(housing)

forest_rmse = mean_squared_error(housing_labels, housing_predictions_forestReg, squared=False)
print('RMSE for Random Forest Regressor: ', forest_rmse.round(2))

RMSE for Random Forest Regressor:  17791.15


## Model Fine-Tuning
Even when a model seems pretty appropriate, it is a good idea to check if it can't be improved by fine-tuning its hyper-parameters.

### Randomized Search
As we saw with the previous example, _GridSearchCV_ is not a scalable options for optimizing hyper-parameters. For this, we will make use of *RandomizedSearchCV*.

In [16]:
params_distrib = {'preprocessing__geo__n_clusters': stats.randint(low=3, high=50),
     'ForestReg__max_features': stats.randint(low=2, high=20)}

rand_search = RandomizedSearchCV(forest_reg, param_distributions=params_distrib, n_iter=10, cv=3, scoring='neg_root_mean_squared_error', random_state=117)
rand_search.fit(housing, housing_labels)

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(remainder=Pipeline(steps=[('impute',
                                                                                           SimpleImputer(strategy='median')),
                                                                                          ('standardize',
                                                                                           StandardScaler())]),
                                                                transformers=[('bedrooms',
                                                                               Pipeline(steps=[('impute',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('computeRatio',
                                                                                                FunctionTransformer(feature_names_out=<function ratio_name a...
                                                                               ['housing_median_age'])])),
                                             ('ForestReg',
                                              RandomForestRegressor(random_state=42))]),
                   param_distributions={'ForestReg__max_features': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f4c29276a40>,
                                        'preprocessing__geo__n_clusters': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f4c29275fc0>},
                   random_state=117, scoring='neg_root_mean_squared_error')

In [17]:
rand_ForestReg = rand_search.best_estimator_
rand_ForestReg

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder=Pipeline(steps=[('impute',
                                                              SimpleImputer(strategy='median')),
                                                             ('standardize',
                                                              StandardScaler())]),
                                   transformers=[('bedrooms',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('computeRatio',
                                                                   FunctionTransformer(feature_names_out=<function ratio_name at 0x7f4c297593f0>,
                                                                                       func=<function col...
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('OneHotEncoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f4c292f12d0>),
                                                 ('age',
                                                  ClusterSimilarity(n_clusters=2,
                                                                    random_state=42),
                                                  ['housing_median_age'])])),
                ('ForestReg',
                 RandomForestRegressor(max_features=4, random_state=42))])

### Analyzing the best models and their errors
Inspecting the resulting models will give us an insight into the relative importance of each attribute when generating the predictions:

In [18]:
final_model = rand_ForestReg
final_featuresImportance = final_model["ForestReg"].feature_importances_
# [str(x.round(2)*100) + '%'for x in final_featuresImportance] #Useless here, since we are going to sort on the values, and that's not possible if we turn our floats into strings :p
sorted(zip(final_featuresImportance, final_model["preprocessing"].get_feature_names_out()), reverse=True)

[(0.13758447969201276, 'log__median_income'),
 (0.06362425700328485, 'rooms_per_house__ratio'),
 (0.062113571376021845, 'cat__ocean_proximity_INLAND'),
 (0.06061210653215922, 'bedrooms__ratio'),
 (0.04697107299145419, 'people_per_house__ratio'),
 (0.04644721780549303, 'geo__Cluster 15 similarity'),
 (0.03608348768357887, 'geo__Cluster 3 similarity'),
 (0.03211445725191155, 'geo__Cluster 2 similarity'),
 (0.030279065339557987, 'geo__Cluster 8 similarity'),
 (0.028917127223013338, 'geo__Cluster 13 similarity'),
 (0.027644254849414022, 'geo__Cluster 27 similarity'),
 (0.02227458757461301, 'geo__Cluster 28 similarity'),
 (0.021840881856364958, 'geo__Cluster 21 similarity'),
 (0.021359360802026477, 'geo__Cluster 22 similarity'),
 (0.021069789112641485, 'geo__Cluster 25 similarity'),
 (0.020670228400098595, 'geo__Cluster 26 similarity'),
 (0.02016418531582129, 'geo__Cluster 17 similarity'),
 (0.020025893268308212, 'geo__Cluster 7 similarity'),
 (0.01925129514174533, 'geo__Cluster 14 similari

Based on this, we can see that only cat__ocean_proximity_INLAND has a relative importance above 1%, so we might want to transform this categorical boolean where INLAND=True and False for all other categories, removing some less useful features.

### Evaluate the predictor on the test set

In [20]:
test_housing = strat_test_set.drop("median_house_value", axis=1)
test_housing_labels = strat_test_set["median_house_value"].copy()

final_predictions = final_model.predict(test_housing)

final_rmse = mean_squared_error(test_housing_labels, final_predictions, squared=False)
print('RMSE for final model, on test data: ', final_rmse.round(2))

RMSE for final model, on test data:  40513.43


So, turns out that even though the fine-tuning granted me much lower RMSE on the training data, this did not translate to significantly improved results when predicting the test data, and I've even remained in the range from the book.

For completeness' sake, lets' compute the 95CI of our estimate:

In [21]:
squared_errors = (final_predictions - test_housing_labels)**2
CI95 = np.sqrt(stats.t.interval(0.95, len(squared_errors)-1, loc=squared_errors.mean(), scale=stats.sem(squared_errors)))
print('95% CI for the RMSE of the final model, on test data: ', CI95.round(2))

95% CI for the RMSE of the final model, on test data:  [38395.63 42525.9 ]


In [22]:
## Attempt 1
# squared_errors_bis = (squared_errors - squared_errors.mean())/stats.sem(squared_errors)
# CI95_bis = np.sqrt(stats.t.interval(0.95, len(squared_errors_bis)-1))
# CI95_bis

## Attempt 2
# float_scale = stats.sem(squared_errors)
# int_df = len(squared_errors)-1
# float_shiftScale_squared_errors = (0.95-squared_errors.mean())/float_scale #TG. Obviously there's an issue here since I'm substracting the mean to 0.95
# CI95_bis = np.sqrt(stats.t.interval(float_shiftScale_squared_errors, int_df)/float_scale)

## Attempt 3
# Oh lol, okay I understand where my confusion comes from!
# int_df = len(squared_errors)-1 #We apply -1 due to the definition of RMSE, but this is just the degree of freedom
# float_SEM = squared_errors.mean() # This is just the average of the squared error. In the following operation, it simply informs stats.t.interval of the position

## Production Tweaks
Enabling tools to simplify launch, update and maintenance of tools is much easier if done from the get go, let's see the most common options presented in the book.
For started, saving the final model is straight forward:

In [23]:
# Let's just cerate a small variable to timestamp correctly our model, to avoid crushing previous one and have better traceability
finalModel_path = "Archive/"+datetime.now().strftime('%Y%m%dT%H%M%S')+"/"

#Create the folder which will contain the data and model archive
os.mkdir(finalModel_path)

with open(finalModel_path+"_final_model.pkl", 'wb') as file:
    joblib.dump(final_model, file)

If we save the model, we should also save the training and test data, so that we are certain we can get them in the exact same shape/form it was during today's session:

In [24]:
with open(finalModel_path+"housing.pkl", 'wb') as file:
    pickle.dump(housing, file)

with open(finalModel_path+"housing_labels.pkl", 'wb') as file:
    pickle.dump(housing_labels, file)

with open(finalModel_path+"test_housing.pkl", 'wb') as file:
    pickle.dump(test_housing, file)

with open(finalModel_path+"test_housing_labels.pkl", 'wb') as file:
    pickle.dump(test_housing_labels, file)


We can then reload the files:

In [25]:
with open(finalModel_path+"test_housing_labels.pkl", 'rb') as file:
    # Call load method to deserialze
    test_housing_labels_reloaded = pickle.load(file)
print(test_housing_labels_reloaded)

14814    285400.0
18689    151300.0
14930    344000.0
5244     158200.0
13323    228100.0
           ...   
20239    300000.0
13623    277500.0
19426    295600.0
1519     115000.0
198       99100.0
Name: median_house_value, Length: 4128, dtype: float64
